In [1]:
import requests
import geojson
import folium
import geopandas
import psycopg2
import psycopg2.extras
import numpy as np
import itertools
import pandas as pd
import rasterio
import rasterio.mask
import gdal
import math
import geoalchemy2
import ogr
import osr
import uuid
import os
# import rasterstats
from collections import Counter
from joblib import Parallel, delayed
from scipy.stats import pareto
from shapely.ops import cascaded_union
from sqlalchemy import create_engine
from rasterstats import zonal_stats
from shapely import wkt
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import MultiPolygon
from shapely.geometry import Polygon
from shapely.wkt import loads

In [ ]:
#Prepare the pareto distribution data

vegetation_dt = geopandas.read_file('/work/veg_source_dt.csv')
structure_dt = geopandas.read_file('/work/str_source_dt.csv')
assembly_dt = geopandas.read_file('/work/asm_source_dt.csv')

veg_pareto_dt = pd.DataFrame(columns=['Wind_Speed', 'Subs','vec_len', 'b','ploc','scale'])
for j, k in itertools.product(['Idle','Medium','High'],['Grass','Shrub','Tree']):
    pareto_vec = vegetation_dt[(vegetation_dt['Vegetation Type']==k) & (vegetation_dt['Wind Speed'] ==j)]['Total Flying Distance  [m]'].values
    pareto_vec = [float(i) for i in pareto_vec]
    b_samp, loc_samp, scale_samp = pareto.fit(pareto_vec)
    n_emb = len(pareto_vec)
    out_dt =[j,k,n_emb,b_samp,loc_samp,scale_samp]
    # veg_pareto_dt = veg_pareto_dt.append(out_dt)
    veg_pareto_dt = veg_pareto_dt.append(
        pd.DataFrame(
            # index=(veg_pareto_dt.index.max()+1,),
            columns=veg_pareto_dt.columns,
            data=(out_dt,),
        )
    )

# print(vegetation_dt)
str_pareto_dt = pd.DataFrame(columns=['Wind_Speed','Assembly','Material','vec_len', 'b','ploc','scale'])

str_simple_dt = structure_dt[['Wind Speed','Assembly','Material']].drop_duplicates()

for i in range(len(str_simple_dt)):
    samp_dt = str_simple_dt.iloc[[i]]
    j=samp_dt['Wind Speed'].values[0]
    k=samp_dt['Assembly'].values[0]
    l=samp_dt['Material'].values[0]
    # print([j,k,l])
    pareto_dt = structure_dt[(structure_dt['Wind Speed']==j) &(structure_dt['Assembly']==k) & (structure_dt['Material']==l)]
    pareto_vec = pareto_dt['Total Flying Distance  [m]'].values
    pareto_vec = [float(m) for m in pareto_vec]
    n_emb = len(pareto_vec)
    b_samp, loc_samp, scale_samp = pareto.fit(pareto_vec)
    out_dt =[j,k,l,n_emb,b_samp,loc_samp,scale_samp]
    # veg_pareto_dt = veg_pareto_dt.append(out_dt)
    str_pareto_dt = str_pareto_dt.append(
            pd.DataFrame(
                # index=(veg_pareto_dt.index.max()+1,),
                columns=str_pareto_dt.columns,
                data=(out_dt,),
            )
        )

asm_pareto_dt = pd.DataFrame(columns=['Wind_Speed','Cladding','Wall Type','Material','Roof (Sheathing)','vec_len', 'b','ploc','scale'])

asm_simple_dt = assembly_dt[['Wind Speed','Cladding','Wall Type','Material','Roof (Sheathing)']].drop_duplicates()

for i in range(len(str_simple_dt)):
    samp_dt = asm_simple_dt.iloc[[i]]
    j=samp_dt['Wind Speed'].values[0]
    k=samp_dt['Cladding'].values[0]
    l=samp_dt['Wall Type'].values[0]
    m=samp_dt['Material'].values[0]
    n=samp_dt['Roof (Sheathing)'].values[0]
    # print([j,k,l])
    pareto_dt = assembly_dt[(assembly_dt['Wind Speed']==j) &(assembly_dt['Cladding']==k) & (assembly_dt['Wall Type']==l)& (assembly_dt['Material']==m)& (assembly_dt['Roof (Sheathing)']==n)]
    pareto_vec = pareto_dt['Total Flying Distance  [m]'].values
    # print(pareto_vec)
    pareto_vec = [float(o) for o in pareto_vec]

    b_samp, loc_samp, scale_samp = pareto.fit(pareto_vec)
    n_emb = len(pareto_vec)
    out_dt =[j,k,l,m,n,n_emb,b_samp,loc_samp,scale_samp]
    # veg_pareto_dt = veg_pareto_dt.append(out_dt)
    asm_pareto_dt = asm_pareto_dt.append(
            pd.DataFrame(
                # index=(veg_pareto_dt.index.max()+1,),
                columns=asm_pareto_dt.columns,
                data=(out_dt,),
            )
        )

/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2606: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py

In [5]:
def get_parcel_geoms(parno,county = 'Sonoma'):
    ## Set up access controlls
    #Todo: should this be some kind of 'secret' or be stored in a secrets thing?
    #Todo: check database to see if a parcel has been queried recently or if we have current enough data
    # and returned that analysis instead of buying another parcel and analyisin
    db_cred = {'user': 'jifesypi',
                'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
                'host': 'mild-williams-pear.db.elephantsql.com',
                'port': '5432', 
                'database': 'jifesypi',
                'epsg': '4326',
                'schema':'schema'
                } 

    #Create connection to the database
    engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

    sql_query_parcels = f"SELECT * FROM public.california_parcels WHERE parno = '{parno}' AND county = '{county}'"

    parcel_dataframe = geopandas.read_postgis(sql_query_parcels, engine)

    #Set parcel number
    # parcel_dataframe= sonoma_parcel_dataframe[sonoma_parcel_dataframe['parno'] == parno]


    print(parcel_dataframe)
    # Query postgres database based on parcel geometry
    #Set query EPSG (WSG 4326)
    parcel_dataframe = parcel_dataframe.set_crs('EPSG:4326')

    #Convert to 26910, stateplane & meters
    parcel_dataframe_26910 = parcel_dataframe.to_crs(26910)

    #Buffer parcel 1000 meters
    buffer_parcel_dataframe_26910 = parcel_dataframe_26910.buffer(1000).to_crs(4326)

    #Extract well known text
    buffer_parcel_geometry_wkt = buffer_parcel_dataframe_26910.geometry.to_wkt().to_list()[0]
    parcel_geometry_wkt = parcel_dataframe_26910.to_crs(4326).geometry.to_wkt().to_list()[0]

    #Query for neighborhood of parcels
    sql_query_parcels = f"SELECT * FROM public.california_parcels WHERE ST_INTERSECTS(geom, ST_BUFFER('SRID=4326;{buffer_parcel_geometry_wkt}', 0))"

    #Send to engine
    neighborhood_parcels = geopandas.read_postgis(sql_query_parcels, engine)

    
    #Need to union to send to wkt query
    # neighborhood_parcels_union = geopandas.GeoSeries(unary_union(neighborhood_parcels.geometry.values))
    neighborhood_parcels_union = neighborhood_parcels.unary_union

    #Get wkt of neighborhood of parcels
    neighborhood_geometry_wkt = neighborhood_parcels_union.wkt

    #Query for extended neighborhood (1000m) of structures
    sql_query_parcels = f"SELECT * FROM public.california_structures WHERE ST_INTERSECTS(geom, ST_BUFFER('SRID=4326;{neighborhood_geometry_wkt}',0))"

    neighborhood_structures = geopandas.read_postgis(sql_query_parcels, engine)

    #Query for parcel structures
    sql_query_structures = f"SELECT * FROM public.california_structures WHERE ST_INTERSECTS(geom, ST_BUFFER('SRID=4326;{parcel_geometry_wkt}',0))"

    parcel_structures = geopandas.read_postgis(sql_query_structures, engine)
    engine.dispose()
    return parcel_dataframe, neighborhood_parcels, parcel_structures, neighborhood_structures

In [6]:
parcel_dataframe, neighborhood_parcels, parcel_structures, neighborhood_structures = get_parcel_geoms('013-400-006',county = 'Sonoma')

        fid  objectid        parno  county address  city   zip  shape_length  \
0  10324826  10614797  013-400-006  Sonoma    None  None  None    932.122019   

     shape_area                                               geom  \
0  25400.526787  MULTIPOLYGON (((-122.68339 38.44932, -122.6833...   

        min_x      min_y       max_x      max_y  
0 -122.686609  38.447894 -122.682828  38.449363  


In [7]:
db_cred = {'user': 'jifesypi',
            'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
            'host': 'mild-williams-pear.db.elephantsql.com',
            'port': '5432', 
            'database': 'jifesypi',
            'epsg': '4326',
            'schema':'schema'
            } 

#Create connection to the database
engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")


neighborhood_parcels_union = neighborhood_parcels.unary_union

#Get wkt of neighborhood of parcels
neighborhood_geometry_wkt = neighborhood_parcels_union.wkt

#Query for neighborhood of parcels
sql_query_parcels = f"SELECT * FROM public.veg_map WHERE ST_INTERSECTS(geom, ST_BUFFER('SRID=4326;{neighborhood_geometry_wkt}', 0))"

#Send to engine
neighborhood_vegetation = geopandas.read_postgis(sql_query_parcels, engine)
engine.dispose()

In [10]:
neighborhood_geometry_wkt

'POLYGON ((-122.69482083415105 38.44120392509451, -122.69506741707099 38.44113738969344, -122.69535398448585 38.4410609201721, -122.69579202907768 38.44093810546171, -122.69603819995395 38.44087066467963, -122.69630668842608 38.440798612672246, -122.69654620940136 38.44073154058792, -122.69660527017383 38.44086079391713, -122.69643794200944 38.4409072492194, -122.69625650446092 38.44095694572557, -122.69609405107951 38.44100202554771, -122.69577206206947 38.44108731817337, -122.69574886698604 38.4410950235118, -122.69573126337274 38.4411051249034, -122.69572145289987 38.44111593442046, -122.69571783251406 38.44112596509512, -122.69571792988387 38.441135959641294, -122.6957227081674 38.44114985278458, -122.69578261838991 38.44128037899116, -122.69586165243398 38.441463787526, -122.69594214404908 38.44163551626927, -122.69602882591272 38.44184227449376, -122.69611812731655 38.442047272418115, -122.69612497023677 38.442059808254676, -122.69613329399024 38.44206666382621, -122.696148289812

In [7]:
def get_vegetation_neighborhood(neighborhood_parcels):
    filename = str(uuid.uuid4())

    neighborhood_parcels_union = neighborhood_parcels.unary_union

    #Get wkt of neighborhood of parcels
    neighborhood_geometry_wkt = neighborhood_parcels_union.wkt

    #Query for neighborhood of parcels
    sql_query_parcels = f"SELECT * FROM public.veg_map WHERE ST_INTERSECTS(geom, ST_BUFFER('SRID=4326;{neighborhood_geometry_wkt}', 0))"

    #Send to engine
    neighborhood_vegetation = geopandas.read_postgis(sql_query_parcels, engine)

    # with rasterio.open("/work/Stack_Sonoma_2021_classification.tif") as src:
    #     class_out_image, class_out_transform = rasterio.mask.mask(src, neighborhood_parcels.geometry, crop=True)
    #     class_out_meta = src.meta

    # class_out_meta.update({"driver": "GTiff",
    #                 "height": class_out_image.shape[1],
    #                 "width": class_out_image.shape[2],
    #                 "transform": class_out_transform})

    with rasterio.open("/work/processing_folder/"+filename+"_class.tif", "w", **class_out_meta) as dest:
        dest.write(class_out_image)


    with rasterio.open("/work/Stack_Sonoma_2021_biomass.tif") as src:
        biomass_out_image, biomass_out_transform = rasterio.mask.mask(src, neighborhood_parcels.geometry, crop=True)
        biomass_out_meta = src.meta

    biomass_out_meta.update({"driver": "GTiff",
                    "height": biomass_out_image.shape[1],
                    "width": biomass_out_image.shape[2],
                    "transform": biomass_out_transform})

    with rasterio.open("/work/processing_folder/"+filename+"_biomass.tif", "w", **biomass_out_meta) as dest:
        dest.write(biomass_out_image)
    # from osgeo import gdal, ogr, osr

    in_path = '/work/processing_folder/'+filename+'_class.tif'

    out_path = '/work/processing_folder/'+filename+'_class.gpkg'

    # #  get raster datasource
    # src_ds = gdal.Open( in_path )
    # #
    # srcband = src_ds.GetRasterBand(1)
    # dst_layername = 'Class'
    # drv = ogr.GetDriverByName("GPKG")
    # dst_ds = drv.CreateDataSource( out_path )

    # sp_ref = osr.SpatialReference()
    # sp_ref.SetFromUserInput('EPSG:4326')

    # dst_layer = dst_ds.CreateLayer(dst_layername, srs = sp_ref )

    # fld = ogr.FieldDefn("Class", ogr.OFTInteger)
    # dst_layer.CreateField(fld)
    # dst_field = dst_layer.GetLayerDefn().GetFieldIndex("Class")

    # gdal.Polygonize( srcband, None, dst_layer, dst_field, [], callback=None )

    # del src_ds
    # del dst_ds

    vegetation_dataframe = neighborhood_vegetation
    
    vegetation_dataframe[['Class_str']] =  vegetation_dataframe[['Class']].values
    d = { 0 : 'Grass' , 4 : 'Grass', 2 : 'Grass', 7 : 'Grass', 5 : 'Shrub', 11 : 'Shrub', 13 : 'Shrub', 14 : 'Shrub',     1 : 'Tree', 3 : 'Tree', 6 : 'Tree', 8 : 'Tree',9 : 'Tree', 10 : 'Tree', 12 : 'Tree',15 : 'Grass'}
    vegetation_dataframe.Class_str = vegetation_dataframe.Class_str.map(d)

    vegetation_dataframe['biomass'] = zonal_stats(vegetation_dataframe, 
                             "/work/processing_folder/"+filename+"_biomass.tif", 
                             stats="mean", # any other stat will also do
                             categorical=False,
                             nodata = np.nan)
    vegetation_dataframe['biomass'] = [i['mean'] for i in vegetation_dataframe['biomass']]
    vegetation_dataframe['area'] = vegetation_dataframe.to_crs(6414).area/10000
    vegetation_dataframe['MG'] = vegetation_dataframe['area']*vegetation_dataframe['biomass']
    # emb_vec = []
    # for i in range(len(vegetation_dataframe)):
    #     iloc_dt = vegetation_dataframe.iloc[[i]]
    #     emb_vec.append(veg_pareto_dt[(veg_pareto_dt.Subs == iloc_dt['Class_str'].values[0] ) &(veg_pareto_dt.Wind_Speed == 'High')].vec_len.values[0])
    # vegetation_dataframe['embers_base'] = emb_vec
    # vegetation_dataframe['embers'] = vegetation_dataframe['MG']*vegetation_dataframe['embers_base']*1000/5


    os.remove("/work/processing_folder/"+filename+"_biomass.tif")
    os.remove('/work/processing_folder/'+filename+'_class.tif')
    os.remove('/work/processing_folder/'+filename+'_class.gpkg')
    return vegetation_dataframe

In [ ]:
parcel_dataframe, neighborhood_parcels, parcel_structures, neighborhood_structures = get_parcel_geoms('013-400-006')

        fid  objectid        parno  county address  city   zip  shape_length  \
0  10324826  10614797  013-400-006  Sonoma    None  None  None    932.122019   

     shape_area                                               geom  \
0  25400.526787  MULTIPOLYGON (((-122.68339 38.44932, -122.6833...   

        min_x      min_y       max_x      max_y  
0 -122.686609  38.447894 -122.682828  38.449363  


In [ ]:
import time

start = time.time()
print("hello")

neighborhood_veg = get_vegetation_neighborhood(neighborhood_parcels)

end = time.time()
print(end - start)

hello
26.347524881362915


In [ ]:
neighborhood_veg

,Class,geometry,Class_str,biomass,area,MG
0,12,"POLYGON ((-122.68894 38.46173, -122.68894 38.4...",Tree,136.000000,0.015637,2.126570
1,1,"POLYGON ((-122.68894 38.46155, -122.68894 38.4...",Tree,134.500000,0.046910,6.309377
2,1,"POLYGON ((-122.68759 38.46155, -122.68759 38.4...",Tree,173.333333,0.023455,4.065512
3,1,"POLYGON ((-122.68741 38.46137, -122.68741 38.4...",Tree,180.000000,0.007818,1.407295
4,6,"POLYGON ((-122.68714 38.46128, -122.68714 38.4...",Tree,170.000000,0.039092,6.645576
...,...,...,...,...,...,...
2553,2,"POLYGON ((-122.68175 38.43855, -122.68175 38.4...",Grass,7.000000,0.023462,0.164236
2554,2,"POLYGON ((-122.68238 38.43846, -122.68238 38.4...",Grass,7.000000,0.007821,0.054745
2555,4,"POLYGON ((-122.68750 38.43837, -122.68750 38.4...",Grass,35.500000,0.015642,0.555274
2556,2,"POLYGON ((-122.68714 38.43936, -122.68714 38.4...",Grass,1079.161290,0.484887,523.271096


In [ ]:
def get_vegetation_pareto(parcel_dataframe):
    filename = str(uuid.uuid4())

    with rasterio.open("/work/Stack_Sonoma_2021_classification.tif") as src:
        class_out_image, class_out_transform = rasterio.mask.mask(src, parcel_dataframe.geometry, crop=True)
        class_out_meta = src.meta

    class_out_meta.update({"driver": "GTiff",
                    "height": class_out_image.shape[1],
                    "width": class_out_image.shape[2],
                    "transform": class_out_transform})

    with rasterio.open("/work/processing_folder/"+filename+"_class.tif", "w", **class_out_meta) as dest:
        dest.write(class_out_image)


    with rasterio.open("/work/Stack_Sonoma_2021_biomass.tif") as src:
        biomass_out_image, biomass_out_transform = rasterio.mask.mask(src, parcel_dataframe.geometry, crop=True)
        biomass_out_meta = src.meta

    biomass_out_meta.update({"driver": "GTiff",
                    "height": biomass_out_image.shape[1],
                    "width": biomass_out_image.shape[2],
                    "transform": biomass_out_transform})

    with rasterio.open("/work/processing_folder/"+filename+"_biomass.tif", "w", **biomass_out_meta) as dest:
        dest.write(biomass_out_image)
    # from osgeo import gdal, ogr, osr

    in_path = '/work/processing_folder/'+filename+'_class.tif'

    out_path = '/work/processing_folder/'+filename+'_class.gpkg'

    #  get raster datasource
    src_ds = gdal.Open( in_path )
    #
    srcband = src_ds.GetRasterBand(1)
    dst_layername = 'Class'
    drv = ogr.GetDriverByName("GPKG")
    dst_ds = drv.CreateDataSource( out_path )

    sp_ref = osr.SpatialReference()
    sp_ref.SetFromUserInput('EPSG:4326')

    dst_layer = dst_ds.CreateLayer(dst_layername, srs = sp_ref )

    fld = ogr.FieldDefn("Class", ogr.OFTInteger)
    dst_layer.CreateField(fld)
    dst_field = dst_layer.GetLayerDefn().GetFieldIndex("Class")

    gdal.Polygonize( srcband, None, dst_layer, dst_field, [], callback=None )

    del src_ds
    del dst_ds

    vegetation_dataframe = geopandas.read_file('/work/processing_folder/'+filename+'_class.gpkg')

    vegetation_dataframe[['Class_str']] =  vegetation_dataframe[['Class']].values
    d = { 0 : 'Grass' , 4 : 'Grass', 2 : 'Grass', 7 : 'Grass', 5 : 'Shrub', 11 : 'Shrub', 13 : 'Shrub', 14 : 'Shrub',     1 : 'Tree', 3 : 'Tree', 6 : 'Tree', 8 : 'Tree',9 : 'Tree', 10 : 'Tree', 12 : 'Tree',15 : 'Grass'}
    vegetation_dataframe.Class_str = vegetation_dataframe.Class_str.map(d)

    vegetation_dataframe['biomass'] = zonal_stats(vegetation_dataframe, 
                             "/work/processing_folder/"+filename+"_biomass.tif", 
                             stats="mean", # any other stat will also do
                             categorical=False,
                             nodata = np.nan)
    vegetation_dataframe['biomass'] = [i['mean'] for i in vegetation_dataframe['biomass']]
    vegetation_dataframe['area'] = vegetation_dataframe.to_crs(6414).area/10000
    vegetation_dataframe['MG'] = vegetation_dataframe['area']*vegetation_dataframe['biomass']
    emb_vec = []
    for i in range(len(vegetation_dataframe)):
        iloc_dt = vegetation_dataframe.iloc[[i]]
        emb_vec.append(veg_pareto_dt[(veg_pareto_dt.Subs == iloc_dt['Class_str'].values[0] ) &(veg_pareto_dt.Wind_Speed == 'High')].vec_len.values[0])
    vegetation_dataframe['embers_base'] = emb_vec
    vegetation_dataframe['embers'] = vegetation_dataframe['MG']*vegetation_dataframe['embers_base']*1000/5


    os.remove("/work/processing_folder/"+filename+"_biomass.tif")
    os.remove('/work/processing_folder/'+filename+'_class.tif')
    os.remove('/work/processing_folder/'+filename+'_class.gpkg')
    return vegetation_dataframe

In [ ]:
def get_structure_pareto(parcel_structures):
    n_embers = str_pareto_dt[(str_pareto_dt.Assembly == 'C' )&(str_pareto_dt.Material == 'CO' ) &(str_pareto_dt.Wind_Speed == 'High')].vec_len[0]
    parcel_structures['embers'] = n_embers*parcel_structures.to_crs(6414).area*6
    return(parcel_structures)

In [ ]:
def structure_zone_gen(i_samp,structure_dataframe):
    iloc_new_dt = structure_dataframe.iloc[[i_samp]]
    b_ = str_pareto_dt[(str_pareto_dt.Assembly == 'C' )&(str_pareto_dt.Material == 'CO' ) &(str_pareto_dt.Wind_Speed == 'High')].b.values[0]
    loc_ = str_pareto_dt[(str_pareto_dt.Assembly == 'C' )&(str_pareto_dt.Material == 'CO' ) &(str_pareto_dt.Wind_Speed == 'High')].ploc.values[0]
    scale_ = str_pareto_dt[(str_pareto_dt.Assembly == 'C' )&(str_pareto_dt.Material == 'CO' ) &(str_pareto_dt.Wind_Speed == 'High')].scale.values[0]

    emb_count =  int(max(iloc_new_dt['embers'].values[0],75000))

    scale = 1

    if emb_count > 1000000:
        scale = emb_count/1000000
        emb_count = 1000000

    emb_dist = pareto.rvs(size=emb_count,b=b_,loc=loc_,scale=scale_)

    emb_dist = [int(math.ceil(i)) for i in emb_dist]


    values, counts = np.unique(emb_dist, return_counts=True)

    counts =  counts*scale

    obj_r = math.sqrt(iloc_new_dt.to_crs(6414).area/3.14) 

    values_area = (3.14*(obj_r+values)*(obj_r+values)) - (3.14*(obj_r+values -1)*(obj_r+values -1)) 

    counts_area = counts/values_area

    max_v_vec = values <= max(values[counts_area > min(1,max(counts_area-1))])

    values = values[max_v_vec]

    counts = counts[max_v_vec]

    counts_area = counts_area[max_v_vec]

    #This is already a short cut. I'd rather not short cut any more than k == 15.
    k = 30
    l = (len(counts)-1)
    k_vec =  np.sort([i for i in range(min(5,len(counts)))]+[l]+[np.ceil(l*(1/(i+1))) for i in range(k)][::-1])
    k_vec = np.sort(list(set(k_vec)))
    k_vec = [int(i) for i in k_vec]
    norm_count_vec = (np.cumsum(counts)[k_vec])/np.sum(counts)
    norm_0_count_vec = np.insert(norm_count_vec, 0, 0)[0:len(norm_count_vec)]
    counts_vec = np.sum(counts)*(norm_count_vec-norm_0_count_vec)
    values_vec = values[k_vec]

    counts, values = counts_vec,values_vec

    xval_low = np.insert(values, 0, 0)[0:(len(values))]
    xval_high = np.insert(values, 0, 0)[1:(len(values)+1)]

    # print(xval_low)
    # print(xval_high)
    def zone_out(xval_low,xval_high):
        zone_low = iloc_new_dt.geometry.buffer(xval_low).unary_union
        zone_high = iloc_new_dt.geometry.buffer(xval_high).unary_union
        return zone_high.difference(zone_low)
    iloc_new_dt = iloc_new_dt.to_crs(6414)
    zone_vec = np.vectorize(zone_out)(xval_low,xval_high)

    zone_vec = np.append(iloc_new_dt.geometry.to_crs(6414),zone_vec)
    counts = np.append(counts[0],counts)
    values = np.append(values[0],values)

    zones = geopandas.GeoDataFrame()
    # zones.is_copy = False

    zones['counts'] = counts
    zones['values'] = values
    zones['values'][0] = 0
    zones.geometry = geopandas.GeoSeries(zone_vec)
    zones['area_r'] =  zones.geometry.area
    counts_area_r_vec = zones['counts']/zones['area_r']
    counts_area_r_vec[0] = counts_area_r_vec[1]
    zones['counts_area_r'] = counts_area_r_vec
    # zones['counts_area_r'][0] = zones['counts_area_r'][1]
    # zones['class'] = np.repeat(iloc_new_dt['Class_str'][0],len(zones['counts_area_r']))
    zones['Class_str'] = 'Structure'
    zones['Class'] = 15
    zones = zones.set_crs('EPSG:6414')
    zones = zones.to_crs(4326)
    return zones

In [ ]:
def veg_zone_gen(i_samp,vegetation_dataframe):
    iloc_new_dt = vegetation_dataframe.iloc[[i_samp]]
    b_ = veg_pareto_dt[(veg_pareto_dt.Subs == iloc_new_dt['Class_str'].values[0] ) &(veg_pareto_dt.Wind_Speed == 'High')].b.values[0]
    loc_ = veg_pareto_dt[(veg_pareto_dt.Subs == iloc_new_dt['Class_str'].values[0] ) &(veg_pareto_dt.Wind_Speed == 'High')].ploc.values[0]
    scale_ = veg_pareto_dt[(veg_pareto_dt.Subs == iloc_new_dt['Class_str'].values[0] ) &(veg_pareto_dt.Wind_Speed == 'High')].scale.values[0]

    emb_count =  int(max(iloc_new_dt['embers'].values[0],75000))

    scale = 1

    if emb_count > 1000000:
        scale = emb_count/1000000
        emb_count = 1000000

    emb_dist = pareto.rvs(size=emb_count,b=b_,loc=loc_,scale=scale_)

    emb_dist = [int(math.ceil(i)) for i in emb_dist]


    values, counts = np.unique(emb_dist, return_counts=True)

    counts =  counts*scale

    obj_r = math.sqrt(iloc_new_dt.to_crs(6414).area/3.14) 

    values_area = (3.14*(obj_r+values)*(obj_r+values)) - (3.14*(obj_r+values -1)*(obj_r+values -1)) 

    counts_area = counts/values_area

    max_v_vec = values <= max(values[counts_area > min(1,max(counts_area-1))])

    values = values[max_v_vec]

    counts = counts[max_v_vec]

    counts_area = counts_area[max_v_vec]

    #This is already a short cut. I'd rather not short cut any more than k == 15.
    k = 30
    l = (len(counts)-1)
    k_vec =  np.sort([i for i in range(min(5,len(counts)))]+[l]+[np.ceil(l*(1/(i+1))) for i in range(k)][::-1])
    k_vec = np.sort(list(set(k_vec)))
    k_vec = [int(i) for i in k_vec]
    norm_count_vec = (np.cumsum(counts)[k_vec])/np.sum(counts)
    norm_0_count_vec = np.insert(norm_count_vec, 0, 0)[0:len(norm_count_vec)]
    counts_vec = np.sum(counts)*(norm_count_vec-norm_0_count_vec)
    values_vec = values[k_vec]

    counts, values = counts_vec,values_vec

    xval_low = np.insert(values, 0, 0)[0:(len(values))]
    xval_high = np.insert(values, 0, 0)[1:(len(values)+1)]

    # print(xval_low)
    # print(xval_high)
    def zone_out(xval_low,xval_high):
        zone_low = iloc_new_dt.geometry.buffer(xval_low).unary_union
        zone_high = iloc_new_dt.geometry.buffer(xval_high).unary_union
        return zone_high.difference(zone_low)
    iloc_new_dt = iloc_new_dt.to_crs(6414)
    zone_vec = np.vectorize(zone_out)(xval_low,xval_high)

    zone_vec = np.append(iloc_new_dt.geometry.to_crs(6414),zone_vec)
    counts = np.append(counts[0],counts)
    values = np.append(values[0],values)

    zones = geopandas.GeoDataFrame()
    # zones.is_copy = False

    zones['counts'] = counts
    zones['values'] = values
    zones['values'][0] = 0
    # print(values)
    # print(xval_low)
    # zones['low'] = xval_low[0:(len(values))]
    # zones['high'] = xval_high[0:(len(values))]


    zones.geometry = geopandas.GeoSeries(zone_vec)
    zones['area_r'] =  zones.geometry.area
    counts_area_r_vec = zones['counts']/zones['area_r']
    counts_area_r_vec[0] = counts_area_r_vec[1]
    zones['counts_area_r'] = counts_area_r_vec
    # zones['counts_area_r'][0] = zones['counts_area_r'][1]
    # zones['class'] = np.repeat(iloc_new_dt['Class_str'][0],len(zones['counts_area_r']))
    zones['Class_str'] = iloc_new_dt['Class_str'].iloc[0]
    zones['Class'] = iloc_new_dt['Class'].iloc[0]


    zones = zones.set_crs('EPSG:6414')
    zones = zones.to_crs(4326)
    return zones

In [ ]:
def summarize_neighborhood(zones,neighborhood_parcels,parcel_dataframe):
    neighborhood_parcels_zones = geopandas.overlay(
        zones,
        neighborhood_parcels,
        how="union")
    neighborhood_parcels_zones = neighborhood_parcels_zones.drop('fid', axis=1)
    neighborhood_parcels_zones['area_r']= neighborhood_parcels_zones.to_crs(6414).area
    neighborhood_parcels_zones['values'] =  neighborhood_parcels_zones['area_r']*neighborhood_parcels_zones['counts_area_r']
    neighborhood_parcels_zones = neighborhood_parcels_zones.groupby(['parno','Class_str']).agg({'values': 'sum','area_r': 'sum'}).reset_index()
    neighborhood_parcels_zones['counts_area_r'] = neighborhood_parcels_zones['values'] /neighborhood_parcels_zones['area_r']
    neighborhood_parcels_results = neighborhood_parcels.merge(neighborhood_parcels_zones, on='parno', how='left')
    neighborhood_parcels_results = neighborhood_parcels_results.drop('fid', axis=1)
    neighborhood_parcels_results = neighborhood_parcels_results.loc[neighborhood_parcels_results['counts_area_r'] >0,:]
    neighborhood_parcels_results['to_parno'] = neighborhood_parcels_results['parno']
    neighborhood_parcels_results['from_parno'] = parcel_dataframe['parno'].iloc[0]
    return neighborhood_parcels_results

In [ ]:
def summarize_neighborhood_kpartite(zones,neighborhood_parcels,parcel_dataframe):

    
    neighborhood_parcels_zones = geopandas.overlay(
        zones,
        neighborhood_parcels,
        how="union")
    neighborhood_parcels_zones = neighborhood_parcels_zones.drop('fid', axis=1)
    neighborhood_parcels_zones['area_r']= neighborhood_parcels_zones.to_crs(6414).area
    neighborhood_parcels_zones['values'] =  neighborhood_parcels_zones['area_r']*neighborhood_parcels_zones['counts_area_r']
    neighborhood_parcels_zones = neighborhood_parcels_zones.groupby(['parno','Class_str']).agg({'values': 'sum','area_r': 'sum'}).reset_index()
    neighborhood_parcels_zones['counts_area_r'] = neighborhood_parcels_zones['values'] /neighborhood_parcels_zones['area_r']
    neighborhood_parcels_results = neighborhood_parcels.merge(neighborhood_parcels_zones, on='parno', how='left')
    neighborhood_parcels_results = neighborhood_parcels_results.drop('fid', axis=1)
    neighborhood_parcels_results = neighborhood_parcels_results.loc[neighborhood_parcels_results['counts_area_r'] >0,:]
    neighborhood_parcels_results['to_parno'] = neighborhood_parcels_results['parno']
    neighborhood_parcels_results['from_parno'] = parcel_dataframe['parno'].iloc[0]
    return neighborhood_parcels_results

In [ ]:
def create_network(parno):
    print(parno)
    if(parno!='999-999-ROW'):
        db_cred = {'user': 'jifesypi',
                'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
                'host': 'mild-williams-pear.db.elephantsql.com',
                'port': '5432', 
                'database': 'jifesypi',
                'epsg': '4326',
                'schema':'schema'
                } 

        #Query for neighborhood of parcels
        sql_query_parcels = f"SELECT from_parno,geom FROM embernet WHERE from_parno = '{parno}'"
        
        engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

        #Send to engine
        does_it_exist = geopandas.read_postgis(sql_query_parcels, engine)
        # does_it_exist = []
        engine.dispose()
        if(len(does_it_exist) == 0):
            print('blah.. doing it')
            parcel_dataframe, neighborhood_parcels, parcel_structures, neighborhood_structures = get_parcel_geoms(parno)


            veg_zone_gen_vec = []
            if(len(parcel_dataframe)>0):
                vegetation_dataframe = get_vegetation_pareto(parcel_dataframe)
                veg_zone_gen_vec = [veg_zone_gen(m,vegetation_dataframe) for m in range(len(vegetation_dataframe['Class']))] 

           
            structure_zone_gen_vec = []
            if(len(parcel_structures)>0):
                structure_dataframe = get_structure_pareto(parcel_structures)
                structure_zone_gen_vec = [structure_zone_gen(m,structure_dataframe) for m in range(len(structure_dataframe))] 


            if((len(structure_zone_gen_vec)> 0) & (len(veg_zone_gen_vec)> 0) ):
                zone_vec = [pd.concat(structure_zone_gen_vec),pd.concat(veg_zone_gen_vec)]

            if((len(structure_zone_gen_vec)== 0) & (len(veg_zone_gen_vec)> 0) ):
                zone_vec = [pd.concat(veg_zone_gen_vec)]

            if((len(structure_zone_gen_vec)> 0) & (len(veg_zone_gen_vec)== 0) ):
                zone_vec = [pd.concat(structure_zone_gen_vec)]

            if((len(structure_zone_gen_vec)== 0) & (len(veg_zone_gen_vec)== 0) ):
                zone_vec = []

            zones = pd.concat(zone_vec)

            if(len(zones)>0):

                neighborhood_parcels_results = summarize_neighborhood(zones,neighborhood_parcels,parcel_dataframe)
                #Create connection to the database
                network_out = neighborhood_parcels_results
                db_cred = {'user': 'jifesypi',
                        'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
                        'host': 'mild-williams-pear.db.elephantsql.com',
                        'port': '5432', 
                        'database': 'jifesypi',
                        'epsg': '4326',
                        'schema':'schema'
                        } 
                engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")
                
                network_out["geom"] = [MultiPolygon([feature]) if isinstance(feature, Polygon) else feature for feature in  network_out["geom"]]
                network_out['geom'] = network_out['geom'].apply(lambda x: geoalchemy2.WKTElement(x.wkt, srid=4326))
                network_out.to_sql('embernet', engine, if_exists='append', index=False, 
                                dtype={'geom': geoalchemy2.Geometry('MultiPolygon', srid= 4326)})
             
                # print(zones)
                zones['parno'] = parno
                zones["geom"] = zones["geometry"]

                zones["geom"] = [MultiPolygon([feature]) if isinstance(feature, Polygon) else feature for feature in zones["geom"]]

                zones.set_geometry("geom")
                zones = zones.drop('geometry', axis=1)

                # zones = zones.loc[zones.geometry.geometry.type=='MultiPolygon']
                zones['geom'] = zones['geom'].apply(lambda x: geoalchemy2.WKTElement(x.wkt, srid=4326))

                zones.to_sql('embernet_zones', engine, if_exists='append', index=False, 
                                dtype={'geom': geoalchemy2.Geometry('MultiPolygon', srid= 4326)})
                engine.dispose()

                return neighborhood_parcels_results
    print(parno+" is naughty, we don't do naughty.")

In [ ]:
def create_riskshed(parno):
    if(parno!='999-999-ROW'):
        db_cred = {'user': 'jifesypi',
                'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
                'host': 'mild-williams-pear.db.elephantsql.com',
                'port': '5432', 
                'database': 'jifesypi',
                'epsg': '4326',
                'schema':'schema'
                } 

        #Query for neighborhood of parcels
        sql_query_parcels = f"SELECT from_parno,to_parno,geom FROM embernet WHERE from_parno = '{parno}'"
        
        engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

        #Send to engine
        does_it_exist = geopandas.read_postgis(sql_query_parcels, engine)

        # sql_query_parcels = f"SELECT parno FROM public.california_parcels"

        # full_parno_vec = geopandas.read_postgis(sql_query_parcels, engine)


        engine.dispose()
        print(len(does_it_exist))
        if(len(does_it_exist)==0):
            neighborhood_parcels_results = create_network(parno)
            parno_vec = [i for i in neighborhood_parcels_results['to_parno'].unique()]

        if(len(does_it_exist)>0):
            parno_vec = [i for i in does_it_exist['to_parno'].unique()]

        # sonoma_parno_vec = [i for i in sonoma_parcel_dataframe['parno']]

        network_to_do_vec = parno_vec
        print(len(network_to_do_vec))
        neighborhood_net = [create_network(x) for x in network_to_do_vec]

In [ ]:
def recover_network(parno):
    if(parno!='999-999-ROW'):
        db_cred = {'user': 'jifesypi',
                'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
                'host': 'mild-williams-pear.db.elephantsql.com',
                'port': '5432', 
                'database': 'jifesypi',
                'epsg': '4326',
                'schema':'schema'
                } 

        #Query for neighborhood of parcels
        sql_query_parcels = f"SELECT * FROM embernet WHERE from_parno = '{parno}' OR to_parno = '{parno}'"
        
        engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

        #Send to engine
        parcel_dt = geopandas.read_postgis(sql_query_parcels, engine)
      
        engine.dispose()
        return parcel_dt
    print(parno+" is naughty, we don't do naughty.")

In [ ]:
def recover_zones(parno_vec):
    zone_list = []
    for parno in parno_vec:
        if(parno!='999-999-ROW'):
            db_cred = {'user': 'jifesypi',
                    'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
                    'host': 'mild-williams-pear.db.elephantsql.com',
                    'port': '5432', 
                    'database': 'jifesypi',
                    'epsg': '4326',
                    'schema':'schema'
                    } 

            #Query for neighborhood of parcels
            sql_query_parcels = f"SELECT * FROM embernet_zones WHERE parno = '{parno}'"
            
            engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

            #Send to engine
            zone_dt = geopandas.read_postgis(sql_query_parcels, engine)
        
            engine.dispose()
            zone_list.append(zone_dt)
    return zone_list
    # print(parno+" is naughty, we don't do naughty.")

In [ ]:
def recover_riskshed(parno):

    network_dt = recover_network(parno)

    parno_vec = [i for i in network_dt['to_parno'].unique()]

    riskshed_net = [recover_network(x) for x in parno_vec]

    riskshed_dt = pd.concat(riskshed_net)
    
    return(riskshed_dt)

In [ ]:
def postprocess_network(network_dt):

    network_dt['to_centroid'] = network_dt['geom'].representative_point()

    from_centroid = []
    for i in network_dt['from_parno']:
        from_centroid.append(network_dt.loc[network_dt.to_parno == i]['to_centroid'].iloc[0])

    from_centroid =  geopandas.GeoSeries(from_centroid)
    network_dt['from_centroid'] = from_centroid

    from_to_line = []
    for i  in range(len(network_dt)):
        from_to_line.append(LineString([network_dt['from_centroid'].iloc[i] , network_dt['to_centroid'].iloc[i]]))
   
    from_to_line = geopandas.GeoSeries(from_to_line)
    network_dt['from_to_line'] = from_to_line
    # ,'to_centroid','from_centroid' 
    # network_dt[["to_parno", "from_parno",'counts_area_r','from_centroid']]
    # network_gpd = geopandas.GeoDataFrame(network_dt[["to_parno", "from_parno",'counts_area_r','from_to_line']], geometry='from_to_line')
    network_dt['to_lat'] = network_dt['to_centroid'].x
    network_dt['to_lon'] = network_dt['to_centroid'].y
    network_dt['from_lat'] = network_dt['from_centroid'].x
    network_dt['from_lon'] = network_dt['from_centroid'].y
    out_network_dt = network_dt[["to_parno", "from_parno",'Class_str','counts_area_r','to_lat','to_lon','from_lat','from_lon','from_to_line']]
    return(network_dt)

In [ ]:
def postprocess_network_kpartite(parno):
    db_cred = {'user': 'jifesypi',
        'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
        'host': 'mild-williams-pear.db.elephantsql.com',
        'port': '5432', 
        'database': 'jifesypi',
        'epsg': '4326',
        'schema':'schema'
        } 
    #Query for neighborhood of parcels
    sql_query_parcels = f"SELECT parno_1, geom FROM embernet_kpartite WHERE parno_1 = '{parno}'"

    engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

    #Send to engine
    does_it_exist = geopandas.read_postgis(sql_query_parcels, engine)

    if(len(does_it_exist) == 0):
        create_network(parno)



    #Query for neighborhood of parcels
    sql_query_parcels = f"SELECT parno_2, geom FROM embernet_kpartite WHERE parno_2 = '{parno}'"

    engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

    #Send to engine
    does_it_exist = geopandas.read_postgis(sql_query_parcels, engine)
    # does_it_exist = []
    engine.dispose()
    if(len(does_it_exist) == 0):
        print('blah.. doing it')

        parcel_dataframe, neighborhood_parcels, parcel_structures, neighborhood_structures = get_parcel_geoms(parno)

        db_cred = {'user': 'jifesypi',
                'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
                'host': 'mild-williams-pear.db.elephantsql.com',
                'port': '5432', 
                'database': 'jifesypi',
                'epsg': '4326',
                'schema':'schema'
                } 

        #Query for neighborhood of parcels
        parcel_wkt = parcel_dataframe.geometry.iloc[0].wkt
        sql_query_parcels = f"SELECT * FROM embernet_zones WHERE ST_Intersects(embernet_zones.geom, ST_GeomFromText('{parcel_wkt}',4326));"
        engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

        #Send to engine
        zones_source = geopandas.read_postgis(sql_query_parcels, engine)

        engine.dispose()

        target_parcel_cover = zones_source[(zones_source['parno'] == parno) & (zones_source['values'] == 0)]
        source_parcel_cover = zones_source[(zones_source['values'] != 0)]
        neighborhood_parcels_zones = geopandas.overlay(
            source_parcel_cover,
            target_parcel_cover,
            how="intersection")

        sample_connection = neighborhood_parcels_zones
        sample_connection['area'] = sample_connection.area
        sample_connection['embers'] = sample_connection['counts_area_r_1']*sample_connection['area']
        sample_summary = sample_connection.groupby(['parno_2','parno_1','Class_str_2','Class_str_1']).agg({'embers': 'sum','area':'sum'}).reset_index()
        sample_summary['embM2'] = sample_summary['embers'] / sample_summary['area'] # simplified the lambda function

        # Get centroids of the target parcel cover by class
        parcel_cover_zones = target_parcel_cover[['parno', 'Class_str', 'geom']].dissolve(by=['Class_str', 'parno']).reset_index()
        parcel_cover_zones['class_centroid'] = parcel_cover_zones.centroid

        # Get centroids of all parcels
        parcel_zones = target_parcel_cover[['parno', 'geom']].dissolve(by=['parno']).reset_index()
        parcel_zones['parcel_centroid'] = parcel_zones.centroid


        # Assign target centroids to each class in the summary
        targ_centroids = dict(zip(parcel_cover_zones['Class_str'], parcel_cover_zones['class_centroid']))
        sample_summary['geom'] = sample_summary['Class_str_2'].apply(lambda x: targ_centroids.get(x))

        parcel_centroids = dict(zip(parcel_zones['parno'], parcel_zones['parcel_centroid']))
        # sample_summary['geom'] = sample_summary['parno_2'].apply(lambda x: parcel_centroids.get(x))

        # zones = zones.loc[zones.geometry.geometry.type=='MultiPolygon']
        sample_summary['geom'] = sample_summary['geom'].apply(lambda x: geoalchemy2.WKTElement(x.wkt, srid=4326))

        # sample_summary['targ_centroid'] = sample_summary['targ_centroid'].apply(lambda x: geoalchemy2.WKTElement(x.wkt, srid=4326))

        # sample_summary['geom'] = sample_summary['parcel_centroid']

        # sample_summary['geom'] = sample_summary['geom'].apply(wkt.loads)

        # sample_summary.set_geometry("geom")

        engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

        sample_summary.to_sql('embernet_kpartite', engine, if_exists='append', index=False, 
                        dtype={'geom': geoalchemy2.Geometry('Point', srid= 4326)})
        engine.dispose()

        # return sample_summary
    print(parno+" is naughty, we don't do naughty.")

# EmberNet API: An Overview

### Introduction

Embernet is a in-progress tool for analyzing risk at an individual parcel, and community of parcels resolution. The basic functionality is to map lab-derived distributions of embers to physical parameters observed from spatial data. These data come together to create a physical network of how, in a given risk scenario, we can expect sets of sources and sinks of risk to interact. These interaction can be mapped over many parcels, creating a network of risk and their subsequent interactions. While there are more extensive frameworks for the precise modeling of the interactions of radiant heat, flames, and embers, these frameworks require extensive computational modeling and a precise understanding of the underlying distribution of spatial patters of features. Embernet offers a middle ground where risk profiles can be developed from spatial data and drive high level management decisions over large spatial areas.

The fundamental unit of analysis in EmberNet is a parcel. Four layers are required to drive EmberNet: a parcel data set, a structure dataset with year of construction, a land cover dataset, and an above ground biomass dataset in the form of megagrams/ hectare. Ember cast distribution normals are taken from Zhou 2019, fitted to an appropriate statistical distribution, and related to the physical quantities identified in the former layers. Ember casts are calculated as one dimensional rays extending from the perimeter of an identified feature. Ember casts are summarized to individual parcels in a 'to' - 'from' manner, allowing for the identification of sources and sinks of risk.

### Operation

The first step to using EmberNet is by identifying an APN of interest:

In [ ]:
parno = '013-400-006'

### create_network('APN')

create_network() creates a half dimensional risk analysis (this is a directed network), where the data for one parcel (APN) is retrieved, and a distribution of embers is created based on the features observed. This function is the base analysis that subsequent functions are developed from. The function returns a set of ember cast distributions per parcel, per feature type (structures, grass, shrubs, or trees) in the form of embers/ m2/ h-1. Current implementation dumps these data into a postgres database for convenient retrieval.

In [ ]:
create_network(parno)

013-400-006
013-400-006 is naughty, we don't do naughty.


### recover_network('APN') 

recover_network() retrieves a single network from the database. This network may be 'degrees' of complete or incomplete depending on what other parcels have been analysed previously. This function will collect all entries where 'to_parno' or 'from_parno' are equal to 'APN'. This function effectively retrieves a 'flat' network of to and from parcel numbers, with their associated geometries and relationships.

In [ ]:
# network_dt = recover_network(parno)

### postprocess_network(network_dt, 'to_from_lines')

postprocess_network() is intended to extend the utility of the recover network function. There are many different ways to visualize or represent networks, and they may require different post-processing options. The goal with this function was to build a place for that to happen. Examples of these options would be linking the network by centroids inside of the component polygons, link the network by the lines connecting these centroids, linking the network for only the polygons themselves, or other post-processing options to support a given analysis. The goal of this function is to abstract those details away to return only what is necessary for a given analysis or visualization. postprocess_network 

In [ ]:
# pp_network_dt = postprocess_network(network_dt)

### create_riskshed('APN') 

create_riskshed() extends the utility of create_network() to derive a network for all of the parcels a given 'APN' connects to as a part of its primary network. This is effectively the 'first order' network, where to and from considerations are made. This has similar behavior to create_network() in that results are logged to a postgres database and require postprocessing.

In [ ]:
# create_riskshed(parno)

### recover_riskshed('APN',order = 1) 

recover_riskshed() is an extension of recover_network() and pulls all of the to - from data from a given 'APN' and the to - from data for its primary connections. This can be extended by 'order' to grab additional data beyond the first order scope of influence.

In [ ]:
# recover_network(parno)

### postprocess_network_kpartite('APN')

postprocess_network_kpartite() returns the connections targeting a given APN, including its self. Data included in this function are any parcels which have previously been process. This function returns any network connections for parcels that have previously had networks generated and does not check for the completeness of the network. This function is the primary network derivation function and should be used after generating the network data for a large regional analysis.

In [ ]:
# create_network(parno)

In [ ]:
# postprocess_network_kpartite(parno)

In [ ]:
import os
path = '/work/CWOs/'
files = os.listdir(path)

file_name = files[1]

CWO_path = path+file_name
CWO_pd = geopandas.read_file(CWO_path)
CWO_pd = CWO_pd.dissolve()
CWO_pd_buffer = CWO_pd.to_crs(6414).buffer(3000).to_crs(4326)
db_cred = {'user': 'jifesypi',
    'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
    'host': 'mild-williams-pear.db.elephantsql.com',
    'port': '5432', 
    'database': 'jifesypi',
    'epsg': '4326',
    'schema':'schema'
    } 

#Query for neighborhood of parcels
district_wkt = CWO_pd_buffer.geometry.iloc[0].wkt
sql_query_parcels = f"SELECT * FROM california_parcels WHERE ST_Intersects(california_parcels.geom, ST_GeomFromText('{district_wkt}',4326));"
engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")
#Send to engine
district_parcels = geopandas.read_postgis(sql_query_parcels, engine)

sql_query_parcels = f"SELECT DISTINCT parno_2,geom FROM embernet_kpartite;"
engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")
#Send to engine
completed_parno = geopandas.read_postgis(sql_query_parcels, engine)

engine.dispose()
district_parcels_left = district_parcels[~district_parcels['parno'].isin(completed_parno['parno_2'].unique())]


In [ ]:
break()

SyntaxError: invalid syntax (759819533.py, line 1)

In [ ]:
print(file_name)

print(len(completed_parno['parno_2'].unique()))


print(len(district_parcels['parno'].unique()))

print(len(district_parcels_left['parno'].unique()))

district_parcels_left['county'].unique()

UpperMarkWestParcels.gpkg
12928
7700
3295


array(['Sonoma', 'Napa'], dtype=object)

In [ ]:
import os
path = '/work/CWOs/'
files = os.listdir(path)
files[2]

'BennetRidgeParcels.gpkg'

In [ ]:
import os
path = '/work/CWOs/'
files = os.listdir(path)

for file_name in files[2:3]:
    CWO_path = path+file_name
    CWO_pd = geopandas.read_file(CWO_path)
    CWO_pd = CWO_pd.dissolve()
    CWO_pd_buffer = CWO_pd.to_crs(6414).buffer(3000).to_crs(4326)
    db_cred = {'user': 'jifesypi',
        'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
        'host': 'mild-williams-pear.db.elephantsql.com',
        'port': '5432', 
        'database': 'jifesypi',
        'epsg': '4326',
        'schema':'schema'
        } 

    #Query for neighborhood of parcels
    district_wkt = CWO_pd_buffer.geometry.iloc[0].wkt
    sql_query_parcels = f"SELECT * FROM california_parcels WHERE ST_Intersects(california_parcels.geom, ST_GeomFromText('{district_wkt}',4326));"
    engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")
    #Send to engine
    district_parcels = geopandas.read_postgis(sql_query_parcels, engine)

    sql_query_parcels = f"SELECT DISTINCT parno_2,geom FROM embernet_kpartite;"
    engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")
    #Send to engine
    completed_parno = geopandas.read_postgis(sql_query_parcels, engine)

    district_parcels = district_parcels[district_parcels['county'] == 'Sonoma']

    engine.dispose()
    district_parcels= district_parcels[~district_parcels['parno'].isin(completed_parno['parno_2'].unique())]

    postprocess_network_kpartite(district_parcels.iloc[1]['parno'])
    print(len(district_parcels))
    # def process(i):
    #     try:
    #         create_network(i)
    #         # postprocess_network_kpartite(i)
    #         result = None
    #     except Exception as e:
    #         print(f"An error occurred for input {i}: {str(e)}")
    #         result = None
    #     return result
        
    # results = Parallel(n_jobs=4)(delayed(process)(i) for i in district_parcels_left['parno'])
    # break
    def process(i):
        try:
            postprocess_network_kpartite(i)           
            result = None
        except Exception as e:
            print(f"An error occurred for input {i}: {str(e)}")
            result = None
        return result
    
    results = Parallel(n_jobs=4)(delayed(process)(i) for i in district_parcels['parno'].unique())


See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://panda

In [0]:
db_cred = {'user': 'jifesypi',
        'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
        'host': 'mild-williams-pear.db.elephantsql.com',
        'port': '5432', 
        'database': 'jifesypi',
        'epsg': '4326',
        'schema':'schema'
        } 

#Query for neighborhood of parcels
district_wkt = ocd_fire_district_buffer.geometry.iloc[0].wkt
sql_query_parcels = f"SELECT * FROM embernet_kpartite;"
engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

#Send to engine
embernet_kpartite = geopandas.read_postgis(sql_query_parcels, engine)

engine.dispose()

In [0]:
embernet_kpartite

In [0]:
combined = np.concatenate((embernet_kpartite['parno_2'].unique(),embernet_kpartite['parno_1'].unique()))
combined = np.unique(combined)
# combined = [elem for elem in combined if "ROW" not in elem]

# Convert unique_arr values to strings and wrap them in single quotes
combined = [f"'{str(v)}'" for v in combined]

# Concatenate the values into a comma-separated string
combined = ', '.join(combined)

In [0]:
db_cred = {'user': 'jifesypi',
        'password': 'Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr',
        'host': 'mild-williams-pear.db.elephantsql.com',
        'port': '5432', 
        'database': 'jifesypi',
        'epsg': '4326',
        'schema':'schema'
        } 

#Query for parcels in network
# sql_query_parcels = f"SELECT * FROM california_parcels WHERE parno IN ({','.join(str(p) for p in combined)});"
sql_query_parcels = f"SELECT * FROM california_parcels WHERE parno IN ({combined})"
engine = create_engine(f"postgresql://{db_cred['user']}:{db_cred['password']}@{db_cred['host']}:{db_cred['port']}/{db_cred['database']}")

#Send to engine
network_parcels = geopandas.read_postgis(sql_query_parcels, engine)

engine.dispose()

In [0]:
import networkx as nx

def make_network(df):
    # create a new graph
    G = nx.DiGraph()

    # create four sets for the four classes of nodes
    structures = set()
    grass = set()
    shrub = set()
    tree = set()

    # add nodes to the graph and to their corresponding sets
    for index, row in df.iterrows():
        G.add_node(row['parno_2'], bipartite=0, node_class=row['Class_str_2'])
        G.add_node(row['parno_1'], bipartite=1, node_class=row['Class_str_1'])
        if row['Class_str_2'] == 'Structure':
            structures.add(row['parno_2'])
        elif row['Class_str_2'] == 'Grass':
            grass.add(row['parno_2'])
        elif row['Class_str_2'] == 'Shrub':
            shrub.add(row['parno_2'])
        elif row['Class_str_2'] == 'Tree':
            tree.add(row['parno_2'])
        if row['Class_str_1'] == 'Structure':
            structures.add(row['Class_str_1'])
        elif row['Class_str_1'] == 'Grass':
            grass.add(row['Class_str_1'])
        elif row['Class_str_1'] == 'Shrub':
            shrub.add(row['Class_str_1'])
        elif row['Class_str_1'] == 'Tree':
            tree.add(row['Class_str_1'])

    # add edges to the graph with edge weights
    for index, row in df.iterrows():
        G.add_edge(row['parno_1'], row['parno_2'], weight=row['embM2'])

    # check the sets of nodes
    print(structures)
    print(grass)
    print(shrub)
    print(tree)

    return G

In [0]:
# embernet_kpartite[(embernet_kpartite['parno_2'] == '075-182-005')  | (embernet_kpartite['parno_1'] == '075-182-005')]
# parno = '075-182-005'
# parno_embernet_kpartite = embernet_kpartite[(embernet_kpartite['parno_2'] == parno)  | (embernet_kpartite['parno_1'] == parno)]
# parno_embernet_kpartite = embernet_kpartite[(embernet_kpartite['parno_2'].isin(parno_embernet_kpartite['parno_2'])) | (embernet_kpartite['parno_2'].isin(parno_embernet_kpartite['parno_2']))]


In [0]:
G = make_network(embernet_kpartite)



In [0]:
for source, target, edge_data in G.edges(data=True):
    print("Source:", source)
    print("Target:", target)
    print("Edge Data:", edge_data)
    print("Embercast Value:", edge_data['embercast_value'])

In [0]:
def is_weighted(graph, weight_attribute='weight'):
    for _, _, edge_data in graph.edges(data=True):
        if weight_attribute in edge_data:
            return True
    return False

print("Is the graph weighted?", is_weighted(G))

import networkx as nx

# Assuming 'G' is your directed graph created with NetworkX DiGraph

# Calculate weighted in-degree for each node
weighted_in_degrees = {node: sum(edge_data['weight'] for _, _, edge_data in G.in_edges(node, data=True)) for node in G.nodes()}

# Calculate weighted out-degree for each node
weighted_out_degrees = {node: sum(edge_data['weight'] for _, _, edge_data in G.out_edges(node, data=True)) for node in G.nodes()}

# Print weighted in-degrees and out-degrees
print("Weighted In-degrees:", weighted_in_degrees)
print("Weighted Out-degrees:", weighted_out_degrees)


In [0]:
embernet_kpartite[embernet_kpartite['parno_2'] == '074-010-036']

In [0]:
import networkx as nx
# from networkx import community
# Compute the community partition using the Louvain method
partition = nx.community.greedy_modularity_communities(G)


In [0]:
partition
# create a dictionary to map nodes to communities
node_to_community = {}
for i, comm in enumerate(partition):
    for node in comm:
        node_to_community[node] = i+1

In [0]:
# node_to_community

In [0]:
from collections import Counter


# Count the frequency of each partition
partition_freq = Counter(node_to_community.values())

# Plot the histogram of partition frequency
plt.hist(partition_freq.values())
plt.xlabel('Community Size')
plt.ylabel('Frequency')
plt.show()

In [0]:
# Join dictionary values to column in GeoDataFrame
network_parcels['community'] = network_parcels['parno'].map(node_to_community)

In [0]:
network_parcels.to_file('/work/test7.gpkg')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5eceab32-5823-49b2-9c95-72828e51fba3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>